In [42]:
import markovify
import pandas as pd
import os
import re
import pylab
import numpy as np
import numpy.random
import matplotlib.pyplot as plt
from PIL import Image

def find_section(word_list, link):
    section=""
    file_names=os.listdir(link)
    for i in file_names:
        f= open(link+i, encoding="utf8")
        text=f.read()
        s=re.search(re.escape(word_list[0])+r"(.*)"+re.escape(word_list[1]),text.replace("\n"," "))
        if not s is None:
            section=section+s.group(1)
        f.close()
    return section

def make_article(reference_link, section_str_dict,section_len_dict):
    whole_article=""
    for i in section_str_dict.keys():
        section_input=""
        for j in section_str_dict.get(i):
            section_input=section_input+find_section(j,reference_link)
        model=markovify.Text(section_input)
        section_output=""
        for t in range(section_len_dict.get(i)):
            section_output=section_output+" "+model.make_sentence()
        if i=="RESULTS":
            if " table" in section_output.lower():
                section_output=section_output+"\n"+pd.DataFrame(np.random.random((5,5))).to_latex()
            if" figure" in section_output.lower():
                x = np.random.randn(5000)
                y = np.random.randn(5000)
                heatmap, xedges, yedges = np.histogram2d(x, y, bins=50)
                plt.clf()
                plt.savefig(os.getcwd()+"\\result_image.png")
                section_output=section_output+"\n"+"\\begin{figure}[H]\n\centering\n\includegraphics*[width=1\\textwidth]{result_image.png}\n\end{figure}"
        whole_article=whole_article+" "+i+"\n"+section_output+"\n"
    return whole_article

In [23]:
with open("C:\\Users\\murat_gga8ya6\\Desktop\\RSG\\hackathon-rsg-paper\\PMC\\PLoS_Comput_Biol\\PLoS_Comput_Biol_2014_Jan_14_10(1)_10_1371_annotation_37bcc34c-7789-4c64-a3cf-902e619f5526.txt") as f:
    text=f.read()

FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\murat_gga8ya6\\Desktop\\RSG\\hackathon-rsg-paper\\PMC\\PLoS_Comput_Biol\\PLoS_Comput_Biol_2014_Jan_14_10(1)_10_1371_annotation_37bcc34c-7789-4c64-a3cf-902e619f5526.txt'

In [22]:
Results_abs=""
link="C:\\Users\\murat_gga8ya6\\Desktop\\RSG\\hackathon-rsg-paper\\PMC\\Bioinformatics\\"
file_names=os.listdir(link)
for i in file_names[:100]:
    f= open(link+i, encoding="utf8")
    text=f.read()
    s=re.search('Results: (.*)',text)
    if not s is None:
        Results_abs=Results_abs+s.group(1)
    f.close()

In [44]:
reference_link="C:\\Users\\murat_gga8ya6\\Desktop\\RSG\\hackathon-rsg-paper\\PMC\\Bioinformatics\\"
section_str_dict={"Motivation":[("Motivation","Results")],"Results":[("Results","Availability")],"Availability":[("Availability","Contact")],"INTRODUCTION":[("INTRODUCTION","RESULTS")],"METHODS":[("METHODS","RESULT"),("METHODS","RESULTS"),("METHOD","RESULT"),("METHOD","RESULTS")],"RESULTS":[("RESULTS","DISCUSSION"),("RESULT","DISCUSSION"),("RESULTS","DISCUSSIONS"),("RESULT","DISCUSSIONS")],"DISCUSSION":[("DISCUSSIONS","CONCLUSION"),("DISCUSSION","CONCLUSION"),("DISCUSSIONS","CONCLUSIONS"),("DISCUSSION","CONCLUSIONS")],"CONCLUSIONS":[("CONCLUSIONS","AVAILABILITY"),("CONCLUSION","ACKNOWLEDGMENTS"),("CONCLUSIONS","FUNDING")]}
section_len_dict={"Motivation":3,"Results":3,"Availability":1,"INTRODUCTION":10,"METHODS":10,"RESULTS":15,"DISCUSSION":5,"CONCLUSIONS":5}
result=make_article(reference_link,section_str_dict,section_len_dict)
print(result)

 Motivation
 The ACSEA approach to the four different countries. Other distance measures depend on this scheme whether the predictive phase, we extract 157 directed PPIs, the set and overall data. However, this may be more complex and overlapping pairs.
 Results
 A heuristic set of possible state transitions on a benchmark simulated dataset, SHARAKU exhibited superior performance is systematically validated on two real applications, namely cataloging neurite morphology of fruit fly VNC and center brain, which have a characteristic pattern of lateral inhibition conducted by Ato-coordinated Notch and EGFR signaling is essential for a read. Moreover, our computational experiments, the GD method outperformed several comparison methods in terms of yj, we update scores iteratively. For attaining far reaching validity, our investigations cover a wide range of versions into a hierarchy of proteins through a number of simulations and a signal transduction network modeling the haematopoietic ste

In [26]:
link="C:\\Users\\murat_gga8ya6\\Desktop\\RSG\\hackathon-rsg-paper\\PMC\\Bioinformatics\\"
file_names=os.listdir(link)
f= open(link+file_names[0], encoding="utf8")
text=f.read()
print(text)
p.close()


==== Front
BioinformaticsbioinfoBioinformaticsBioinformatics1367-48031367-4811Oxford University Press 1723707310.1093/bioinformatics/btl650Applications NotesDatabases and OntologiesProServer: a simple, extensible Perl DAS server Finn Robert D. Stalker James W. Jackson David K. Kulesha Eugene Clements Jody Pettett Roger *Wellcome Trust Sanger Institute, Wellcome Trust Geome Campus, Hinxton, Cambridge, CB10 1SA, UK*To whom correspondence should be addressed.Associate Editor: Alfonso Valencia

15 6 2007 18 1 2007 23 12 1568 1570 © 2007 The Author(s)This is an Open Access article distributed under the terms of the Creative Commons Attribution Non-Commercial License (http://creativecommons.org/licenses/by-nc/2.0/uk/) which permits unrestricted non-commercial use, distribution, and reproduction in any medium, provided the original work is properly cited.Summary: The increasing size and complexity of biological databases has led to a growing trend to federate rather than duplicate them. In o

NameError: name 'p' is not defined

In [26]:
model=markovify.Text(Results_abs)
for i in range(5):
    print(model.make_sentence())

It also offers a wide range of visualization functions for structural similarity searching, clustering of residue types and evolutionary conservation.
Remaining recognition problems are to detect exons in several genes confirmed by reverse transcriptase PCR.We discuss the algorithms and various utilities for managing complex compound data.
This approach allows for a bipartite graph, the lack of need for 3D structure information of the underlying combinatorial problem, our method on both simulated and real time series expression data.
Comparative analysis will be critically helpful for this problem, which is based on weighted fragments and genotypes with errors.
The latter monitor the gene expression data in yeast.


In [27]:
Motivation_abs=""
link="C:\\Users\\murat_gga8ya6\\Desktop\\RSG\\hackathon-rsg-paper\\PMC\\Bioinformatics\\"
file_names=os.listdir(link)
for i in file_names[:100]:
    f= open(link+i, encoding="utf8")
    text=f.read()
    s=re.search('Motivation: (.*)',text)
    if not s is None:
        Motivation_abs=Motivation_abs+s.group(1)
    f.close()

In [29]:
model=markovify.Text(Motivation_abs)
for i in range(5):
    print(model.make_sentence())

They can as well be applied to the forefront as a structural one, assuming a fixed backbone.
It would be to distribute the individual comparisons on a genome is complicated by specific features of these resources to assess the improvement in prediction achieved when such information is taken into account.
Unfortunately, peptide fragmentation is complex and not fully understood, and what is understood is not scalable to very large datasets.Recently, structural genomic variants have come to the disease.
Clearly, certain choices of probability distributions will be important for correctly controlling FDR.
New methods are useful for predicting the location of gene expression, and other DNA-related processes.


In [40]:
intro=""
link="C:\\Users\\murat_gga8ya6\\Desktop\\RSG\\hackathon-rsg-paper\\PMC\\Bioinformatics\\"
file_names=os.listdir(link)
for i in file_names[:100]:
    f= open(link+i, encoding="utf8")
    text=f.read()
    s=re.search('INTRODUCTION\n(.*)',text)
    if not s is None:
        intro=intro+s.group(1)
    f.close()

In [79]:
section=""
link="C:\\Users\\murat_gga8ya6\\Desktop\\RSG\\hackathon-rsg-paper\\PMC\\Bioinformatics\\"
file_names=os.listdir(link)
for i in file_names[:100]:
    f= open(link+i, encoding="utf8")
    text=f.read()
    s=re.search("(.*)",text.replace("\n"," "))
    if not s is None:
        section=section+s.group(1)
    f.close()

In [56]:
intro=""
link="C:\\Users\\murat_gga8ya6\\Desktop\\RSG\\hackathon-rsg-paper\\PMC\\Bioinformatics\\"
file_names=os.listdir(link)
for i in file_names[:100]:
    f= open(link+i, encoding="utf8")
    text=f.read()
    s=re.search('==== Body(.*)==== Refs',text.replace("\n"," "))
    if not s is None:
        intro=intro+s.group(1)
    f.close()

In [70]:
np.unique([i for i in re.findall("([A-Z]+)",intro) if len(i)> 5])

array(['AAAAACCTTCATTTTATTTTATCTCCA', 'AAACACATCAGCAGCTTCAATACC',
       'AAAGGTTCAACT', 'AATAAA', 'AATAAACA', 'AATACGTAC', 'AAUAAA',
       'ACAACA', 'ACAACACTTAA', 'ACAACACTTAAAGGTTCAACTAGCAATA',
       'ACATTCCAGCCCCAAAACC', 'ACCOUNTS', 'ACKNOWLEDEGMENTS',
       'ACKNOWLEDGEMENT', 'ACKNOWLEDGEMENTS', 'ACKNOWLEDGMENTS',
       'ACTTTTGAGTTAGGCCCTTTTT', 'ADMINISTRATION', 'AGACGTG',
       'AGCAGCTCGTAATGCATCCT', 'AGGTAAG', 'AGGTAAGT', 'AGMIAL', 'AGTAGT',
       'AGTAGTG', 'AGTCAGTT', 'ALGORITHM', 'ALGORITHMIC', 'ALGORITHMS',
       'ALIGNING', 'ANALYSIS', 'APPENDIX', 'APPLICATION', 'APPLICATIONS',
       'APPROACH', 'APPROXMGD', 'AREVIR', 'ARGUMENTATION', 'ARGUMENTION',
       'ARGUMENTS', 'ASPECTS', 'ASTRAL', 'ATATAAA', 'ATATTAT', 'ATGTGCTA',
       'AUGIST', 'AVAILABILITY', 'BACKBONE', 'BACKGROUND', 'BACKRUB',
       'BASELINE', 'BATWING', 'BEHAVIOR', 'BENCHMARKING', 'BERNWARD',
       'BIGINT', 'BINARY', 'BIOACTIVITY', 'BIOLOGICAL', 'BIOLOGISTS',
       'BIOLOGY', 'BIOMEDICAL', 'B